In [13]:
import numpy as np
from sympy import *

G = str(input("Enter group name as 'Capital letter + number', e.g. A2,A4,F4,G2,E6,..."))

rankG = int((lie.Lie_rank(G))) #rank of the group

def fullchar(v, g):
    '''This function returns the full character, with arguments weight and group.'''
    return lie.W_orbit(lie.dom_char(v,g),g)


def coeffs(p):
    '''This function takes a numpy polynomial argument and returns its coefficients as a list of integers'''
    length = int(lie.length(p))
    coeffs =[int(lie.coef(p,i+1)) for i in range(length)]
    return coeffs


def exponents(p):
    '''This function takes a numpy polynomial argument and returns its exponents as a list of integers'''
    length = int(lie.length(p))
    return [list(map(int,lie.expon(p,i+1))) for i in range(length)]

fund_weights_list = [] #List which will hold the fundamental weights.
for i in range(rankG):
    x = [int(0) for j in range(rankG)] #create zero list
    x[i] = int(1) #put a 1 in ith position
    fund_weights_list.append(x)

fund_polys_coeff_list = []
fund_polys_exponent_list = []
for k in range(len(fund_weights_list)):
    p = fullchar(fund_weights_list[k],G)
#if you want the fundamental characters printed uncomment the below line.
    #print("fundamental character for "+str(fund_weights_list[k])+" is "+str(p))
    fund_polys_coeff_list.append(coeffs(p))
    fund_polys_exponent_list.append(exponents(p))

class LengthError(Exception): #exception for later when input length is not the right size
    pass

go_again = 'y'

while go_again != 'n':
    while True:
        try:
            Rval = [int(item) for item in input("Enter the value of H at each root, order them as in Humphrey's (Care: in Collingwood McGovern order of roots for G2 is different) each seperated by a space: ").split()]
            if len(Rval) != rankG:
               raise LengthError
            break
        except LengthError:
            print("You have input the wrong number of integers, please insert "+str(rankG)+" integers, each seperated by space")
        except ValueError:
            print("One or more entries were not of type int. Please try again")

#We now have a list of integers telling us how H acts on roots, however we need to know how H acts on fundamental weights, to make this conversion we use the cartan matrix, or rather it's inverse, (weight values)^T = C^{-1}(root values)^T

    C_LiEElement = lie.Cartan(G) #Cartan matrix of G above, note that this is of type LiEElement so indexing of matrix starts from 1, not 0. We will fix this.
    C = [] #This will be our Cartan matrix, where entries will be lists containing type int, i.e. converting all the type LiEElement objects
    for i in range(1,rankG+1): 
        row = []
        for j in range(1, rankG+1):
            row.append(int(C_LiEElement[i][j]))
        C.append(row)
    C_inverse = np.linalg.inv(C)

    Wval_Numpy = C_inverse.dot(np.array(Rval)) #this will be a list of values telling us how H acts on the fundamental weights, entries are type numpy.float64 I think.

    Wval = []
    for i in range(0,rankG):
        Wval.append(float(Wval_Numpy[i]))

#if you want to know how H acts on roots/weights/both uncomment below
    #print("values of H on roots are "+str(Rval))
    #print("values of H on weights are "+str(Wval))

#Restrict the character of each fundamental representation to SL2, these polynomials are in terms of \gamma, not t, where t = \gamma + \gamma^{-1} is the 2d irrep of SL2.

    res_poly_master_list = []    
    for k in range(len(fund_weights_list)):
        p = fullchar(fund_weights_list[k],G)
        restriction_exponents = [int(round(sum(Wval[j]*fund_polys_exponent_list[k][i][j] for j in range(rankG)))) for i in range(int(lie.length(p)))]
        
        #Now we have two lists, coeff_list which gives a list of coeffs from character polynomial, and restriction_exponents, a list of exponents
        #this may look something like [1,3,2,3] and [2,-2,2,0] i.e. the poly 1x^2 + 3x^{-2}+2x^2+3, observe that x^2 appears twice and the exponents are not in particular order in the latter list, now we adjust these lists to get 3x^2 + 3 + 3x^{-2} i.e [3,3,3] and [-2,0,-2]
        M = max(restriction_exponents)
        n = min(restriction_exponents)
        coeffs_ordered = []
        restricted_ordered = []
        for i in range(M,n-1,-1): #run through each exponent from max to min decreasing by 1 each loop
            num_appearances = 0
            for j in range(len(restriction_exponents)):
                if i == restriction_exponents[j]:
                    num_appearances = num_appearances + fund_polys_coeff_list[k][j]
            coeffs_ordered.append(num_appearances)
            restricted_ordered.append(i)
        res_poly_master_list.append([coeffs_ordered,restricted_ordered])

    t = symbols('t') #indeterminate 't' for our restricted polynomials. Note R(SL2) = Z[t], t = \gamma + \gamma^-1 is the 2d irrep.
    init_printing(use_unicode=False, wrap_line=False)

    sl2_irrep_polys = [1,t] #e.g. where , P_0 = 1, P_1 = x and use [P_1][P_n] = [P_{n-1}]+[P_{n+1}] to calculate all them (or as many as we need)

    max_irrep = 0 #the maximum P_i that we will need, based on the maximum exponent of the restricted character polynomials.
    for i in range(rankG):
        if res_poly_master_list[i][1][0] >= max_irrep:
            max_irrep = res_poly_master_list[i][1][0]

    for i in range(2,max_irrep+1):
        X = sl2_irrep_polys[1]*sl2_irrep_polys[i-1] - sl2_irrep_polys[i-2]
        sl2_irrep_polys.append(expand(X))

#sl2_irrep_polys is a list of polynomials P_0, P_1. Indexing is nice as sl2_irrep_polys[k] = P_k, corresponding to representation with highest weight k.

    char_polys_wrt_t_coeffs = [] #this will be a list of lists of integers with number of each P_i appearing in each fundamental representation
    for i in range(rankG): #filling char_polys_wrt_t_coeffs with rankG zero lists, each of length max_irreps to hold coefficients of P_0,P_1,..,etc
        zeros = [int(0) for j in range(max_irrep+1)]
        char_polys_wrt_t_coeffs.append(zeros)


    #The next loop decomposes our restricted characters into sum of the P_i i.e. decompose into sum of sl2 irreps
    #Warning: This for loop wipes res_poly_master_list[i][0][j] for all i,j so can't call it again
    for i in range(rankG):
        for j in range(res_poly_master_list[i][1][0]+1):
            if res_poly_master_list[i][0][j] != 0:
                subtract = res_poly_master_list[i][0][j]
                char_polys_wrt_t_coeffs[i][res_poly_master_list[i][1][j]] += res_poly_master_list[i][0][j]
                for k in range(res_poly_master_list[i][1][j]+1):
                    res_poly_master_list[i][0][j+2*k] = res_poly_master_list[i][0][j+2*k] - subtract
    
    char_polys_wrt_t = [] #these are numpy.polynomial.Polynomials w.r.t t.

    for i in range(rankG):
        f = 0
        for j in range(max_irrep+1):
            f += char_polys_wrt_t_coeffs[i][j]*sl2_irrep_polys[j]
        char_polys_wrt_t.append(f)
    
    for i in range(rankG):
        print("")
        print("Restricted character to SL2 - dimension of representation #"+str(i+1)+" is ")
        print(str(char_polys_wrt_t[i]-int(lie.dim(fund_weights_list[i],G)))+" which factors as "+str(factor(char_polys_wrt_t[i] - int(lie.dim(fund_weights_list[i],G)))))

    print("")
    print("Would you like to input another weighted Dynkin diagram? (anything for yes / 'n' for no)")
    go_again = str(input())

Enter group name as 'Capital letter + number', e.g. A2,A4,F4,G2,E6,... G2
Enter the value of H at each root, order them as in Humphrey's (Care: in Collingwood McGovern order of roots for G2 is different) each seperated by a space:  2 2



Restricted character to SL2 - dimension of representation #1 is 
t**6 - 5*t**4 + 6*t**2 - 8 which factors as (t - 2)*(t + 2)*(t**4 - t**2 + 2)

Restricted character to SL2 - dimension of representation #2 is 
t**10 - 9*t**8 + 28*t**6 - 35*t**4 + 16*t**2 - 16 which factors as (t - 2)*(t + 2)*(t**8 - 5*t**6 + 8*t**4 - 3*t**2 + 4)

Would you like to input another weighted Dynkin diagram? (anything for yes / 'n' for no)


KeyboardInterrupt: Interrupted by user